This is the final project with all of the experiments stripped out.

### load modules

In [16]:
import sys
import pickle
sys.path.append("../tools/")

import pandas as pd
import numpy as np

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

import matplotlib.pyplot as plt
%matplotlib inline

### import data into Pandas DataFrame

In [17]:
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
df = pd.DataFrame(data_dict)
df = df.transpose()

### Task 1: Select Features

In [18]:
df = df.drop('email_address', axis=1)
df = df.astype(float)

### Task 2: Eliminate Outliers/Missing Data

In [19]:
df = df.drop('TOTAL')
df = df.drop("THE TRAVEL AGENCY IN THE PARK")
df.fillna(0, inplace=True)

### Task 3: Create New Features

In [20]:
df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)

COACH QUESTION: I have put this into the machine learning models but I don't know how to evaluate how useful they were. It is not like regression where you have a coefficient and t-score. What is the procedure for evaluating a particular feature? 

#### Select K best features
Here I am going to try to weed out some features before the features list is submitted to the testing program. If it were not for the Udacity testing program it would make sense to do this at a later point or maybe in the pipeline. 

In [21]:
y = df['poi']
X = df.drop('poi', axis=1)

In [31]:
from sklearn.feature_selection import SelectKBest
X_new = SelectKBest().fit_transform(X, y)

In [35]:
X_new

array([[  4.17500000e+06,  -3.08105500e+06,   1.72954100e+06, ...,
          1.40700000e+03,   4.48444200e+06,   1.72954100e+06],
       [  0.00000000e+00,   0.00000000e+00,   2.57817000e+05, ...,
          0.00000000e+00,   1.82466000e+05,   2.57817000e+05],
       [  0.00000000e+00,  -5.10400000e+03,   4.04615700e+06, ...,
          4.65000000e+02,   9.16197000e+05,   5.24348700e+06],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   1.39130000e+05, ...,
          0.00000000e+00,   0.00000000e+00,   1.39130000e+05],
       [  0.00000000e+00,   0.00000000e+00,   8.30855200e+06, ...,
          0.00000000e+00,   3.60300000e+05,   1.18847580e+07],
       [  0.00000000e+00,   0.00000000e+00,   1.92758000e+05, ...,
          0.00000000e+00,   5.50970000e+04,   1.92758000e+05]])

COACH QUESTION: What does it mean "Input X must be non-negative"? X is not negative. There are some places where it is zero. 

How do I do the k-best algorithm? In general, having trouble doing things while working with the grader. Should I do all the testing outside the context of the grader and only submit the final model to the grader? I am so confused by having to use the grader. Is it just me? 

ANSWER: I was using the chi2 which is for categorical data. 

#### sklearn prep: features_list
Put dependent variable at the top

In [23]:
features_list = list(df.columns)
features_list.remove('poi')
features_list.insert(0, 'poi')

#### Udacity prep: change to dictionary

In [24]:
df = df.transpose()
data_dict = df.to_dict()
### Store to my_dataset for easy export below.
my_dataset = data_dict
### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)
### Run through Udacity's grading script
from tester import test_classifier
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)

### Task 4: Try a variety of classifiers
Done in other notebook
### Task 5: Tune precision and recall > 0.3

#### Standardize by subtracting mean and dividing by std_dev

In [25]:
from sklearn import preprocessing
# Create a minimum and maximum processor object
standard_scaler = preprocessing.StandardScaler()

# Create an object to transform the data to fit minmax processor
features_train = standard_scaler.fit_transform(features_train)
features_test = standard_scaler.fit_transform(features_test)

Now I am going to use the select k best method on the data and then see how it does with the models. 

COACH QUESTION: Where do I do the k-best thing? Can it be added to a pipeline below? 

In [36]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
test_classifier(clf, my_dataset, features_list, folds = 1000)

GaussianNB()
	Accuracy: 0.61633	Precision: 0.21514	Recall: 0.70900	F1: 0.33011	F2: 0.48592
	Total predictions: 15000	True positives: 1418	False positives: 5173	False negatives:  582	True negatives: 7827



COACH QUESTION: Why doesn't the rbf kernel work? 

In [29]:
from sklearn import svm
clf = svm.SVC(kernel='rbf',C=100) 
# tried gamma=0.001, C=100.
# kernel = 'linear', kernel='rbf'
test_classifier(clf, my_dataset, features_list, folds=1000)

Got a divide by zero when trying out: SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Precision or recall may be undefined due to a lack of true positive predicitons.


In [30]:
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA(n_components=7)), ('svm', svm.SVC(kernel='rbf'))]
clf = Pipeline(estimators)
test_classifier(clf, my_dataset, features_list, folds = 1000)

Got a divide by zero when trying out: Pipeline(steps=[('reduce_dim', PCA(copy=True, n_components=7, whiten=False)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
Precision or recall may be undefined due to a lack of true positive predicitons.


### The Winner
COACH QUESTION: So the Guassian Naive Bayes with the dimensions of the data set reduced to 8 through prinicple component analysis is the one that I got to break the 0.3 thresholds in Recall and Precision. I did it by just doing a manual search through the possible number of components. I have no idea which components were chosen and on what basis they worked, which 8 worked better than 9 or why 7 didn't make the 0.3 cut at all. I have no intuition for why any of this worked. And I don't know how to visualize it. Can I just submit this for the final project? 

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA(n_components=8)), ('Gaussian', GaussianNB())]
clf = Pipeline(estimators)
test_classifier(clf, my_dataset, features_list, folds = 1000)

Pipeline(steps=[('reduce_dim', PCA(copy=True, n_components=8, whiten=False)), ('Gaussian', GaussianNB())])
	Accuracy: 0.85053	Precision: 0.42568	Recall: 0.34650	F1: 0.38203	F2: 0.35989
	Total predictions: 15000	True positives:  693	False positives:  935	False negatives: 1307	True negatives: 12065



In [37]:
clf.params_

AttributeError: 'Pipeline' object has no attribute 'params_'